<a href="https://colab.research.google.com/github/yonatanrtt/NER/blob/main/NLP_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers --quiet

     |████████████████████████████████| 365 kB 29.0 MB/s 
     |████████████████████████████████| 4.7 MB 75.7 MB/s 
     |████████████████████████████████| 212 kB 105.2 MB/s 
     |████████████████████████████████| 115 kB 85.9 MB/s 
     |████████████████████████████████| 120 kB 91.3 MB/s 
     |████████████████████████████████| 127 kB 95.9 MB/s 
     |████████████████████████████████| 6.6 MB 87.0 MB/s 


In [2]:
import pandas as pd

data = pd.read_csv('./demo_bot_intents.csv')

In [3]:
from datasets import Dataset

def conv_to_ds(_data, _is_train):
  filtered_df = _data[_data['is_train']==int(_is_train)][['text', 'lables']]
  filtered_df = filtered_df.rename(columns={"lables": "labels"})
  return Dataset.from_pandas(filtered_df)

In [4]:
train_dataset  = conv_to_ds(data, _is_train=True)
test_dataset = conv_to_ds(data, _is_train=False)

In [5]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(
        model_name,
        num_labels=5
        )
model = AutoModelForSequenceClassification.from_config(config)

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="./tests_trainer")

In [8]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./test_trainer", evaluation_strategy="epoch")

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 40
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15
